In [37]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from random import randint
from warnings import warn
from IPython.core.display import clear_output

from time import time
from datetime import datetime
from nba_api.stats.endpoints import leaguegamelog

/var/folders/9_/skd397mj5fzg2y5xx2wb5xfc0000gn/T/ipykernel_63006/2646622729.py:7: DeprecationWarning: Importing clear_output from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import clear_output


In [38]:
url = 'https://www.covers.com/sports/NBA/matchups?selectedDate=2020-03-11'

response = get(url)
print(response.text[:500])


<!DOCTYPE html>
<html lang="en" xml:lang="en">
<head>

    


    <title>NBA Matchups 2019-2020 - Today&#39;s Basketball Previews, Scores, &amp; Schedules</title>
    <meta name="description" content="NBA Scores &amp; Matchups for Mar 11, 2020 including previews, scores, schedule, stats, results, betting trends, and more.">
    <meta name="keywords" content="nba, nba scores, nba matchups, basketball, nba games, nba results, nba logros , escore nba, resultados nba">
    <meta name="co


In [39]:
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

In [40]:
game_containers = html_soup.find_all('div', class_ = 'cmg_matchup_game_box cmg_game_data')
print(type(game_containers))
print(len(game_containers))

<class 'bs4.element.ResultSet'>
4


In [41]:
games = html_soup.find_all('tbody')
for game in games:
    print(game.text.split())

['DET', '29', '26', '29', '22', '106', 'Odds', 'PHI', '34', '36', '29', '25', '124', 'Odds']
['NY', '33', '34', '29', '22', '18', '136', 'Odds', 'ATL', '24', '26', '28', '40', '13', '131', 'Odds']
['CHA', '22', '37', '28', '22', '109', 'Odds', 'MIA', '40', '11', '26', '21', '98', 'Odds']
['DEN', '29', '21', '35', '12', '97', 'Odds', 'DAL', '33', '24', '27', '29', '113', 'Odds']


In [42]:
first = games[0]

In [43]:
#Extract Quarter Scores/Totals
first_game = game_containers[0]
scores = first_game.find('div', class_ = 'cmg_matchup_line_score')
scores = scores.find_all('tr')

q1_pts_away = 0 
q1_pts_home = 0
q2_pts_away = 0
q2_pts_home = 0
q3_pts_away = 0
q3_pts_home = 0
q4_pts_away = 0
q4_pts_home = 0
ot1_pts_away = 0
ot1_pts_home = 0
ot2_pts_away = 0
ot2_pts_home = 0
ot3_pts_away = 0
ot3_pts_home = 0
ot4_pts_away = 0
ot4_pts_home = 0

quarters_away = [q1_pts_away, q2_pts_away, q3_pts_away, q4_pts_away, ot1_pts_away, ot2_pts_away, ot3_pts_away, ot4_pts_away]
quarters_home = [q1_pts_home, q2_pts_home, q3_pts_home, q4_pts_home, ot1_pts_home, ot2_pts_home, ot3_pts_home, ot4_pts_home]

scores_list_away = scores[1].text.split()[1:-1]
for i in range(len(scores_list_away)):
    quarters_away[i] = scores_list_away[i]

scores_list_home = scores[2].text.split()[1:-1]
for i in range(len(scores_list_home)):
    quarters_home[i] = scores_list_home[i]
    
    
q1_pts_away, q2_pts_away, q3_pts_away, q4_pts_away, ot1_pts_away, ot2_pts_away, ot3_pts_away, ot4_pts_away = tuple(quarters_away)
q1_pts_home, q2_pts_home, q3_pts_home, q4_pts_home, ot1_pts_home, ot2_pts_home, ot3_pts_home, ot4_pts_home = tuple(quarters_home)

In [44]:
# Extract spreads (for home team) and over/unders
game_data = html_soup.find_all('div', class_ = 'cmg_game_data')
first_bet = game_data[0]

away_team_abbr = first_bet['data-away-team-shortname-search']
home_team_abbr = first_bet['data-home-team-shortname-search']
event_id = first_bet['data-event-id']
home_score = first_bet['data-home-score']
away_score = first_bet['data-away-score']
spread = first_bet['data-game-odd']
over_under = first_bet['data-game-total']

print(away_team_abbr, home_team_abbr, event_id, home_score, away_score, spread, over_under)

DET PHI 1007658 124 106 -12 210


In [45]:
# intialize lists to store data

home_team_abbr = []
away_team_abbr = []
gameID = []
home_score = []
away_score = []
spread = []
over_under = []
q1_pts_away = [] 
q1_pts_home = []
q2_pts_away = []
q2_pts_home = []
q3_pts_away = []
q3_pts_home = []
q4_pts_away = []
q4_pts_home = []
ot1_pts_away = []
ot1_pts_home = []
ot2_pts_away = []
ot2_pts_home = []
ot3_pts_away = []
ot3_pts_home = []
ot4_pts_away = []
ot4_pts_home = []
date = []

In [46]:
type(game_containers[3].find('td', class_='cmg_matchup_line_score_total'))

bs4.element.Tag

In [47]:
for game in game_containers:
    if game.find('td', class_ ='cmg_matchup_line_score_total') is not None:
        quarter_scores = game.find('div', class_ = 'cmg_matchup_line_score')
        quarter_scores = quarter_scores.find_all('tr')
        
        # initialize these variables to 0 so that I can store them as a list (there is probably a better way to do this)
        q1_pts_a = 0 
        q1_pts_h = 0
        q2_pts_a = 0
        q2_pts_h = 0
        q3_pts_a = 0
        q3_pts_h = 0
        q4_pts_a = 0
        q4_pts_h = 0
        ot1_pts_a = 0
        ot1_pts_h = 0
        ot2_pts_a = 0
        ot2_pts_h = 0
        ot3_pts_a = 0
        ot3_pts_h = 0
        ot4_pts_a = 0
        ot4_pts_h = 0

        quarters_away = [q1_pts_a, q2_pts_a, q3_pts_a, q4_pts_a, ot1_pts_a, ot2_pts_a, ot3_pts_a, ot4_pts_a]
        quarters_home = [q1_pts_h, q2_pts_h, q3_pts_h, q4_pts_h, ot1_pts_h, ot2_pts_h, ot3_pts_h, ot4_pts_h]

        scores_list_away = quarter_scores[1].text.split()[1:-1]     # this removes the team_name and the final score, since I already have those and just leaves the quarter scores
        
        for i in range(len(scores_list_away)):
            quarters_away[i] = scores_list_away[i]

        scores_list_home = quarter_scores[2].text.split()[1:-1]
        for i in range(len(scores_list_home)):
            quarters_home[i] = scores_list_home[i]

        # store them into these variables
        q1_pts_a, q2_pts_a, q3_pts_a, q4_pts_a, ot1_pts_a, ot2_pts_a, ot3_pts_a, ot4_pts_a = tuple(quarters_away)
        q1_pts_h, q2_pts_h, q3_pts_h, q4_pts_h, ot1_pts_h, ot2_pts_h, ot3_pts_h, ot4_pts_h = tuple(quarters_home)
        
        q1_pts_away.append(q1_pts_a)
        q2_pts_away.append(q2_pts_a)
        q3_pts_away.append(q3_pts_a)
        q4_pts_away.append(q4_pts_a)
        ot1_pts_away.append(ot1_pts_a)
        ot2_pts_away.append(ot2_pts_a)
        ot3_pts_away.append(ot3_pts_a)
        ot4_pts_away.append(ot4_pts_a)
        
        q1_pts_home.append(q1_pts_h)
        q2_pts_home.append(q2_pts_h)
        q3_pts_home.append(q3_pts_h)
        q4_pts_home.append(q4_pts_h)
        ot1_pts_home.append(ot1_pts_h)
        ot2_pts_home.append(ot2_pts_h)
        ot3_pts_home.append(ot3_pts_h)
        ot4_pts_home.append(ot4_pts_h)
  
        away_team_abbr.append(game['data-away-team-shortname-search'])
        home_team_abbr.append(game['data-home-team-shortname-search'])
        gameID.append(game['data-event-id'])
        home_score.append(game['data-home-score'])
        away_score.append(game['data-away-score'])
        spread.append(game['data-game-odd'])
        over_under.append(game['data-game-total'])
        date.append(game['data-game-date'])
        

In [48]:
betting_data = pd.DataFrame({'event_id':event_id, 
                             'date':date,
                             'home_team_abbr':home_team_abbr,
                             'away_team_abbr':away_team_abbr, 
                             'home_score' : home_score,
                             'away_score' : away_score,
                             'q1_pts_home': q1_pts_home,
                             'q2_pts_home': q2_pts_home,
                             'q3_pts_home': q3_pts_home,
                             'q4_pts_home': q4_pts_home,
                             'ot1_pts_home': ot1_pts_home,
                             'ot2_pts_home': ot2_pts_home,
                             'ot3_pts_home': ot3_pts_home,
                             'ot4_pts_home': ot4_pts_home,
                             'q1_pts_away': q1_pts_home,
                             'q2_pts_away': q2_pts_home,
                             'q3_pts_away': q3_pts_home,
                             'q4_pts_away': q4_pts_home,
                             'ot1_pts_away': ot1_pts_home,
                             'ot2_pts_away': ot2_pts_home,
                             'ot3_pts_away': ot3_pts_home,
                             'ot4_pts_away': ot4_pts_home,
                             'spread' : spread,
                             'total' : over_under
                            })

In [49]:
betting_data

,event_id,date,home_team_abbr,away_team_abbr,home_score,away_score,q1_pts_home,q2_pts_home,q3_pts_home,q4_pts_home,...,q1_pts_away,q2_pts_away,q3_pts_away,q4_pts_away,ot1_pts_away,ot2_pts_away,ot3_pts_away,ot4_pts_away,spread,total
0,1007658,2020-03-11 19:00:00,PHI,DET,124,106,34,36,29,25,...,34,36,29,25,124,0,0,0,-12,210
1,1007658,2020-03-11 19:30:00,ATL,NY,131,136,24,26,28,40,...,24,26,28,40,13,131,0,0,-4.5,232
2,1007658,2020-03-11 19:30:00,MIA,CHA,98,109,40,11,26,21,...,40,11,26,21,98,0,0,0,-11,206.5
3,1007658,2020-03-11 20:00:00,DAL,DEN,113,97,33,24,27,29,...,33,24,27,29,113,0,0,0,3,221


SCRAPE WHOLE PAGE

In [53]:
from warnings import warn
warn("Warning Simulation")
from IPython.core.display import clear_output
from time import time
start_time = time()

requests = 0
import numpy as np
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.library.parameters import SeasonAll

/var/folders/9_/skd397mj5fzg2y5xx2wb5xfc0000gn/T/ipykernel_63006/1872411503.py:2: UserWarning: Warning Simulation
  warn("Warning Simulation")
/var/folders/9_/skd397mj5fzg2y5xx2wb5xfc0000gn/T/ipykernel_63006/1872411503.py:3: DeprecationWarning: Importing clear_output from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import clear_output


In [59]:
seasons = ['20{}-{}'.format(x, x+1) for x in range(10, 24)]
dates = np.array([])
for season in seasons:
    games = leaguegamelog.LeagueGameLog(season=season).get_data_frames()[0]
    dates = np.append(dates, games['GAME_DATE'].unique())

print(len(dates))
print(dates[0], dates[-1])
dates = dates.tolist()[:]

2210
2010-10-26 2024-04-14


In [60]:
# intialize lists to store data

home_team_abbr = []
away_team_abbr = []
event_id = []
home_score = []
away_score = []
spread = []
total = []
q1_pts_away = [] 
q1_pts_home = []
q2_pts_away = []
q2_pts_home = []
q3_pts_away = []
q3_pts_home = []
q4_pts_away = []
q4_pts_home = []
ot1_pts_away = []
ot1_pts_home = []
ot2_pts_away = []
ot2_pts_home = []
ot3_pts_away = []
ot3_pts_home = []
ot4_pts_away = []
ot4_pts_home = []
game_date = []

# To help control loop rate
start_time = time()
requests = 0

for date in dates:
    url = 'https://www.covers.com/sports/NBA/matchups?selectedDate={}'.format(date)        #url for every day in the 2015-2020 season (there will be repeats on days that have no game. we can delete these after)
    # Make a request
    response = get(url)
    
    # Pause the Loop
    sleep(randint(2, 4))
    
    #Monitor the requests
    requests+=1
    elapsed_time = time() - start_time
    print('Request: {}; Frequency: {}'.format(requests, requests/elapsed_time))
    clear_output(wait=True)
    
    # Warning for non-200 status codes
    if response.status_code != 200:
        warn('request: {}; Status code: {}'.format(requests, response.status_code))
        
    # Break loop if number of requests is greater than expetcted
#     if requests > 300:
#         warn('Number of requests was greater than expected')
#         break
    
    html_soup = BeautifulSoup(response.text, 'html.parser')
    
    game_containers = html_soup.find_all('div', class_ = 'cmg_matchup_game_box cmg_game_data')

    for game in game_containers:
        if game.find('td', class_ ='cmg_matchup_line_score_total') is not None:
            quarter_scores = game.find('div', class_ = 'cmg_matchup_line_score')
            quarter_scores = quarter_scores.find_all('tr')

            # initialize these variables to 0 so that I can store them as a list (there is probably a better way to do this)
            q1_pts_a = 0 
            q1_pts_h = 0
            q2_pts_a = 0
            q2_pts_h = 0
            q3_pts_a = 0
            q3_pts_h = 0
            q4_pts_a = 0
            q4_pts_h = 0
            ot1_pts_a = 0
            ot1_pts_h = 0
            ot2_pts_a = 0
            ot2_pts_h = 0
            ot3_pts_a = 0
            ot3_pts_h = 0
            ot4_pts_a = 0
            ot4_pts_h = 0

            quarters_away = [q1_pts_a, q2_pts_a, q3_pts_a, q4_pts_a, ot1_pts_a, ot2_pts_a, ot3_pts_a, ot4_pts_a]
            quarters_home = [q1_pts_h, q2_pts_h, q3_pts_h, q4_pts_h, ot1_pts_h, ot2_pts_h, ot3_pts_h, ot4_pts_h]
            
            # this removes the team_name and the final score, since I already have those and just leaves the quarter scores
            scores_list_away = quarter_scores[1].text.split()[1:-1]     

            for i in range(len(scores_list_away)):
                quarters_away[i] = scores_list_away[i]

            scores_list_home = quarter_scores[2].text.split()[1:-1]
            for i in range(len(scores_list_home)):
                quarters_home[i] = scores_list_home[i]

            # store them into these variables
            q1_pts_a, q2_pts_a, q3_pts_a, q4_pts_a, ot1_pts_a, ot2_pts_a, ot3_pts_a, ot4_pts_a = tuple(quarters_away)
            q1_pts_h, q2_pts_h, q3_pts_h, q4_pts_h, ot1_pts_h, ot2_pts_h, ot3_pts_h, ot4_pts_h = tuple(quarters_home)

            q1_pts_away.append(q1_pts_a)
            q2_pts_away.append(q2_pts_a)
            q3_pts_away.append(q3_pts_a)
            q4_pts_away.append(q4_pts_a)
            ot1_pts_away.append(ot1_pts_a)
            ot2_pts_away.append(ot2_pts_a)
            ot3_pts_away.append(ot3_pts_a)
            ot4_pts_away.append(ot4_pts_a)

            q1_pts_home.append(q1_pts_h)
            q2_pts_home.append(q2_pts_h)
            q3_pts_home.append(q3_pts_h)
            q4_pts_home.append(q4_pts_h)
            ot1_pts_home.append(ot1_pts_h)
            ot2_pts_home.append(ot2_pts_h)
            ot3_pts_home.append(ot3_pts_h)
            ot4_pts_home.append(ot4_pts_h)

            away_team_abbr.append(game['data-away-team-shortname-search'])
            home_team_abbr.append(game['data-home-team-shortname-search'])
            event_id.append(game['data-event-id'])
            home_score.append(game['data-home-score'])
            away_score.append(game['data-away-score'])
            spread.append(game['data-game-odd'])
            total.append(game['data-game-total'])
            game_date.append(date)

Request: 2210; Frequency: 0.2582322125111023


In [61]:
nba_betting_data = pd.DataFrame({'event_id':event_id, 
                             'date':game_date,
                             'home_team_abbr':home_team_abbr,
                             'away_team_abbr':away_team_abbr, 
                             'home_score' : home_score,
                             'away_score' : away_score,
                             'q1_pts_home': q1_pts_home,
                             'q2_pts_home': q2_pts_home,
                             'q3_pts_home': q3_pts_home,
                             'q4_pts_home': q4_pts_home,
                             'ot1_pts_home': ot1_pts_home,
                             'ot2_pts_home': ot2_pts_home,
                             'ot3_pts_home': ot3_pts_home,
                             'ot4_pts_home': ot4_pts_home,
                             'q1_pts_away': q1_pts_home,
                             'q2_pts_away': q2_pts_home,
                             'q3_pts_away': q3_pts_home,
                             'q4_pts_away': q4_pts_home,
                             'ot1_pts_away': ot1_pts_home,
                             'ot2_pts_away': ot2_pts_home,
                             'ot3_pts_away': ot3_pts_home,
                             'ot4_pts_away': ot4_pts_home,
                             'spread' : spread,
                             'total' : total
                            })

In [65]:
def generate_game_id(date_str, home_team_abbreviation):
    # Convert date string to datetime object
    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
    formatted_date = date_obj.strftime('%Y%m%d')
    # Combine date and home team abbreviation
    return f"{formatted_date}0{home_team_abbreviation}"

In [63]:
nba_betting_data.to_csv('data/nba_spreads.csv', index=False)

In [68]:
hi

NameError: name 'hi' is not defined

In [70]:
nba_betting_data['gameID'] = nba_betting_data.apply(lambda row: generate_game_id(row['date'], row['home_team_abbr']), axis=1)

In [78]:
selected_columns = nba_betting_data[['gameID', 'spread', 'total']]

In [75]:
selected_columns.to_csv('data/nba_spreads_totals.csv', index=False)

In [79]:
team_abbreviation_mapping = {
    'NY': 'NYK',
    'BK': 'BRK',
    'MIA': 'MIA',
    'HOU': 'HOU',
    'PHI': 'PHI',
    'NYK': 'NYK',
    'IND': 'IND',
    'POR': 'POR',
    'ORL': 'ORL',
    'MEM': 'MEM',
    'NO': 'NOP',
    'OKC': 'OKC',
    'SAC': 'SAC',
    'DEN': 'DEN',
    'DAL': 'DAL',
    'BOS': 'BOS',
    'SA': 'SAS',
    'PHO': 'PHX',
    'ATL': 'ATL',
    'GS': 'GSW',
    'LAL': 'LAL',
    'TOR': 'TOR',
    'CHA': 'CHA',
    'DET': 'DET',
    'UTA': 'UTA',
    'WAS': 'WAS',
    'MIL': 'MIL',
    'CHI': 'CHI',
    'CLE': 'CLE',
    'MIN': 'MIN'
}

# Function to replace the incorrect abbreviation in the gameID
def update_gameid_with_abbreviation(gameID):
    for incorrect_abbr, correct_abbr in team_abbreviation_mapping.items():
        if incorrect_abbr in gameID:
            return gameID.replace(incorrect_abbr, correct_abbr)
    return gameID  # Return the original gameID if no replacement is made

# Apply the function to the gameID column
selected_columns['gameID'] = selected_columns['gameID'].apply(update_gameid_with_abbreviation)

# Save the updated dataset to a new CSV file
selected_columns.to_csv('data/nba_spreads_totals.csv', index=False)

/var/folders/9_/skd397mj5fzg2y5xx2wb5xfc0000gn/T/ipykernel_63006/301183879.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns['gameID'] = selected_columns['gameID'].apply(update_gameid_with_abbreviation)
